# Includes & prerequisites

In [1]:
!cp /content/drive/MyDrive/Colab\ Notebooks/include/tuning_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/models.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/sca_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/data_utils.py /content/
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [2]:
from tuning_utils import CleanMemoryTuner, model_builder_synthetic_se_cnn, vgg_cbam_synthetic_builder
from models import cbam_block, vgg_cnn_cbam, cnn_squeeze_excite
from data_utils import SyntheticDataLoader

# Train the model with best hyperparameters on the full RPA dataset

In [3]:
model_config = {
    "CBAM": {
        "tuner_dir": "/content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_datasets/CBAM/hypertuning",
        "model_builder_fun": vgg_cbam_synthetic_builder,
        "model_fun": vgg_cnn_cbam
    },
    "SE": {
        "tuner_dir": "/content/drive/MyDrive/mbedtls_synthetic_traces/hyperparameter_tuning_efficient",
        "model_builder_fun": model_builder_synthetic_se_cnn,
        "model_fun": cnn_squeeze_excite
    }
}

In [4]:
# ----- Configuration -----

MODEL = "CBAM"
BYTE = 2

H5_FILE_PATH = f"/content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_datasets/profiling_windows/byte_{BYTE}.h5"

RUN_ROOT = "/content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_training"
BATCH_SIZE = 128
EPOCHS = 60
VALIDATION_SPLIT = 0.2
# LR = 1e-3
SEED = 1917

In [5]:
# ----- Reproducibility -----

import os, json, random, numpy as np, tensorflow as tf
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [6]:
# ----- Paths

from datetime import datetime
run_id = datetime.now().strftime(f"B{BYTE}_{MODEL}_%Y%m%d_%H%M%S")

RUN_DIR = f"{RUN_ROOT}/runs_{MODEL}{run_id}"
CKPT_BEST = f"{RUN_DIR}/best.keras"
LOG_JSON = f"{RUN_DIR}/config.json"
os.makedirs(RUN_DIR, exist_ok=True)

## Load the training data

Based on the `H5_FILE_PATH`, load the appropriate training data (in this case the RPA dataset).

In [7]:
h5_loader = SyntheticDataLoader(file_path=H5_FILE_PATH)
ds_train, ds_full_val, INPUT_DIM, train_size, val_size = h5_loader.create_dataset(
    window_index=0,
    batch_size=BATCH_SIZE,
    val_split=VALIDATION_SPLIT
)

Dataset created from 'byte_2.h5':
  - Total samples: 25001
  - Training samples: 20001
  - Validation samples: 5000


In [8]:
ds_val_for_fit = ds_full_val.map(lambda trace, label, pt, key: (trace, label))

## Load the model from tuning directory

Get the best hyperparameters from the tuning runs on a random dataset and rebuild the model.

In [9]:
tuner = CleanMemoryTuner(
    hypermodel=lambda hp: model_config[MODEL]['model_builder_fun'](
            hp,
            model_fun=model_config[MODEL]['model_fun'],
            input_dim=INPUT_DIM
        ),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    directory=model_config[MODEL]['tuner_dir'],
    project_name=f"tuning_byte_{BYTE}",
    overwrite=False
)

Reloading Tuner from /content/drive/MyDrive/mbedtls_synthetic_traces/RANDOM_datasets/CBAM/hypertuning/tuning_byte_2/tuner0.json


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"best_hps: {best_hps.values}")
model = tuner.hypermodel.build(best_hps)

best_hps: {'filter_scale': 1.5, 'dropout': 0.3, 'cbam_ratio': 32, 'batch_normalize': True, 'optimizer': 'adam', 'learning_rate': 5e-05}


In [11]:
model.summary()

Model: "cbam_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 701, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 701, 96)   │      1,152 │ input_layer[0][0] │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 701, 96)   │        384 │ block1_conv1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool1        │ (None, 350, 96)   │          0 │ batch_normalizat… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 350, 192)  │    202,944 │ block1_pool1[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 350, 192)  │        768 │ block2_conv1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 192)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 192)       │          0 │ batch_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 6)         │      1,158 │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 192)       │      1,344 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 192)       │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_1[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 192)       │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 192)    │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 350, 192)  │          0 │ batch_normalizat… │
│                     │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ channelwise_pool    │ (None, 350, 2)    │          0 │ multiply[0][0]    │
│ (ChannelwisePool)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 350, 1)    │         14 │ channelwise_pool… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 350, 192)  │          0 │ multiply[0][0],   │
│ (Multiply)          │                   │            │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 94,692,666 (361.22 MB)

 Trainable params: 94,688,250 (361.21 MB)

 Non-trainable params: 4,416 (17.25 KB)

In [12]:
print(model.optimizer.learning_rate)

<Variable path=adam/learning_rate, shape=(), dtype=float32, value=4.999999873689376e-05>


## Run the training

In [13]:
csv_log = tf.keras.callbacks.CSVLogger(f"{RUN_DIR}/train_log.csv")
cbs = [
    tf.keras.callbacks.ModelCheckpoint(CKPT_BEST, monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True),
    csv_log
]

In [14]:
print("\n--- Starting Model Training on HDF5 Data ---")
history = model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_val_for_fit,
    callbacks=cbs,
    verbose=2
)


--- Starting Model Training on HDF5 Data ---
Epoch 1/60


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


157/157 - 55s - 348ms/step - accuracy: 0.0039 - loss: 5.5462 - val_accuracy: 0.0050 - val_loss: 5.5471
Epoch 2/60
157/157 - 43s - 272ms/step - accuracy: 0.0047 - loss: 5.5425 - val_accuracy: 0.0060 - val_loss: 5.5470
Epoch 3/60
157/157 - 63s - 401ms/step - accuracy: 0.0055 - loss: 5.5396 - val_accuracy: 0.0060 - val_loss: 5.5300
Epoch 4/60
157/157 - 63s - 402ms/step - accuracy: 0.0059 - loss: 5.5333 - val_accuracy: 0.0088 - val_loss: 5.5122
Epoch 5/60
157/157 - 63s - 402ms/step - accuracy: 0.0070 - loss: 5.5171 - val_accuracy: 0.0148 - val_loss: 5.4795
Epoch 6/60
157/157 - 63s - 400ms/step - accuracy: 0.0121 - loss: 5.4723 - val_accuracy: 0.0278 - val_loss: 5.3852
Epoch 7/60
157/157 - 63s - 400ms/step - accuracy: 0.0230 - loss: 5.3549 - val_accuracy: 0.0660 - val_loss: 5.1456
Epoch 8/60
157/157 - 63s - 400ms/step - accuracy: 0.0533 - loss: 5.0676 - val_accuracy: 0.2014 - val_loss: 4.6432
Epoch 9/60
157/157 - 63s - 401ms/step - accuracy: 0.1489 - loss: 4.4459 - val_accuracy: 0.4410 - va

In [15]:
model.save(f"{RUN_DIR}/final.keras")

In [16]:
print([l.numpy() for _, l in ds_train.unbatch().batch(10)])

[array([  5,  96,  91,  44, 200,  83,  69,   6, 236, 221], dtype=int32), array([165, 136, 216, 215,  77, 199, 200, 177, 225, 202], dtype=int32), array([  8,  33, 195, 175, 206, 148,  77, 142, 106,  22], dtype=int32), array([154,  35,  68, 108, 200,  91, 227, 162, 171,   7], dtype=int32), array([  0, 176, 205, 127,  95, 150, 193, 151,   9, 238], dtype=int32), array([ 38,  54, 196,   8, 156,  24, 144, 247, 124, 140], dtype=int32), array([236, 194,  98, 251, 249, 234, 240, 243, 235,  88], dtype=int32), array([105,  98,  71, 147,  34,  94,  13, 142,  30,  32], dtype=int32), array([ 17, 152, 214,  76, 166, 164,  42,  31, 106,  93], dtype=int32), array([ 13,  42, 222,  74, 131, 241, 121,  39, 254,  10], dtype=int32), array([  8, 161, 239,  84,  41, 107,  33, 127, 101,   1], dtype=int32), array([ 19, 173,   3,  90, 228, 106, 148,  71, 185,  87], dtype=int32), array([ 28, 247, 160, 112, 167,  65, 124, 112,  49, 224], dtype=int32), array([  3, 195, 107, 165, 116, 217,  90, 226, 220, 197], dtype

In [17]:
ds_val_X = ds_full_val.map(lambda trace,label,pt,key: trace)
raw = model.predict(ds_val_X, verbose=0)

FROM_LOGITS = True

if FROM_LOGITS:
  val_logits = raw
else:
  val_logits = tf.math.log(tf.clip_by_value(raw, 1e-12, 1.0)).numpy()

val_labels = np.array([int(lbl.numpy()) for _, lbl, _, _ in ds_full_val.unbatch()], dtype=np.int64)

np.savez(f"{RUN_DIR}/val_logits_labels.npz", logits=val_logits, labels=val_labels)